In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
from keys import password

In [2]:
url = "https://www.imdb.com/chart/top-english-movies?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=BHKD7EHF9EAN90MKN8QN&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_4"

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/pardodo/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [4]:
browser.visit(url)

In [5]:
html = browser.html

In [6]:
soup = bs(html,"html.parser")

In [7]:
titles = soup.find_all(class_="titleColumn")

In [8]:
movie_title = [movie.find("a").text for movie in titles]

In [9]:
 movie_year = [movie.find("span").text for movie in titles]

In [10]:
rating = soup.find_all(class_="ratingColumn imdbRating")

In [11]:
movie_rating = [rate.find("strong").text for rate in rating]

In [12]:
ranking = [i for i in range(1,251)]

In [13]:
striped_year = []
for i in movie_year:
    i = i.replace("(","")
    i = i.replace(")","")
    striped_year.append(i)

In [14]:
imdb = pd.DataFrame(list(zip(ranking, movie_title, striped_year, movie_rating)), columns=["Ranking","Movie Title","Year","Rating"])

In [15]:
imdb

,Ranking,Movie Title,Year,Rating
0,1,The Shawshank Redemption,1994,9.2
1,2,The Godfather,1972,9.2
2,3,The Godfather: Part II,1974,9.0
3,4,The Dark Knight,2008,9.0
4,5,12 Angry Men,1957,9.0
...,...,...,...,...
245,246,The King's Speech,2010,8.0
246,247,Rio Bravo,1959,8.0
247,248,Annie Hall,1977,8.0
248,249,Slumdog Millionaire,2008,8.0


In [16]:
top_4k = pd.read_csv("top_4000_movies_data.csv")
top_4k = top_4k.dropna()
top_4k = top_4k[top_4k["Domestic Gross"] != 0]
top_4k = top_4k[top_4k["Worldwide Gross"] != 0]

In [17]:
top_4k

,Release Date,Movie Title,Production Budget,Domestic Gross,Worldwide Gross
0,4/23/2019,Avengers: Endgame,400000000,858373000,2797800564
1,5/20/2011,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802
2,4/22/2015,Avengers: Age of Ultron,365000000,459005868,1395316979
3,12/16/2015,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817
4,4/25/2018,Avengers: Infinity War,300000000,678815482,2044540523
...,...,...,...,...,...
3995,6/13/2067,You Only Live Twice,9500000,43100000,111600000
3996,6/10/1988,Poltergeist III,9500000,14114000,14114000
3997,12/21/2016,Dangal,9500000,12391761,305149461
3998,3/19/2010,The Runaways,9500000,3573673,5278632


In [18]:
merged_df = imdb.merge(top_4k, on="Movie Title")
merged_df

,Ranking,Movie Title,Year,Rating,Release Date,Production Budget,Domestic Gross,Worldwide Gross
0,1,The Shawshank Redemption,1994,9.2,9/23/1994,25000000,28241469,28419106
1,3,The Godfather: Part II,1974,9.0,12/11/1974,13000000,57300000,57300000
2,4,The Dark Knight,2008,9.0,7/11/2008,185000000,533720947,998615789
3,7,The Lord of the Rings: The Return of the King,2003,8.9,12/17/2003,94000000,377845905,1120214046
4,9,The Lord of the Rings: The Fellowship of the Ring,2001,8.8,12/19/2001,93000000,315544750,891216824
...,...,...,...,...,...,...,...,...
125,242,12 Monkeys,1995,8.0,12/27/1995,29000000,57141459,168841459
126,244,Her,2013,8.0,1/10/2014,23000000,25568251,48043998
127,245,La La Land,2016,8.0,11/30/2016,20000000,151101803,444535865
128,249,Slumdog Millionaire,2008,8.0,11/12/2008,14000000,141330703,383825427


In [19]:
rds_connection_string = f"postgres:{password}@localhost:5432/project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')
if not database_exists(engine.url):
    create_database(engine.url)

In [20]:
imdb.to_sql(name='imdb', con=engine, if_exists='replace', index=False)

In [21]:
top_4k.to_sql(name='top_4k', con=engine, if_exists='replace', index=False)

In [22]:
merged_df.to_sql(name='merged_df', con=engine, if_exists='replace', index=False)

In [23]:
browser.quit()